# Definir os caminhos (padrão novo)

In [0]:

BASE = "abfss://imoveis@datalakealura.dfs.core.windows.net"
INBOUND = f"{BASE}/inbound"
BRONZE  = f"{BASE}/bronze"

ARQUIVO_INBOUND = f"{INBOUND}/dados_brutos_imoveis.json"
PATH_BRONZE = f"{BRONZE}/dataset_imoveis"




# Validar que o arquivo existe no inbound

In [0]:
dbutils.fs.ls(INBOUND)


# Ler o JSON

In [0]:
df = spark.read.json(ARQUIVO_INBOUND)
display(df)


# Remover colunas irrelevantes (imagens, usuario)

In [0]:
df_anuncio = df.drop("imagens", "usuario")
display(df_anuncio)


# Criar a coluna id a partir de anuncio.id

In [0]:
from pyspark.sql.functions import col

df_bronze = df_anuncio.withColumn("id", col("anuncio.id").cast("string"))
display(df_bronze)


# Salvar em Delta na camada Bronze (overwrite)

In [0]:
(df_bronze.write
 .format("delta")
 .mode("overwrite")
 .save(PATH_BRONZE))


# Validar gravação (listar e ler de volta)

In [0]:
dbutils.fs.ls(PATH_BRONZE)


In [0]:
bronze_lido = spark.read.format("delta").load(PATH_BRONZE)
display(bronze_lido)
